In [ ]:

!pip install torch==1.5.1+cu101 torchvision==0.6.1+cu101 -f https://download.pytorch.org/whl/torch_stable.html
from google.colab import drive
from torch.utils.tensorboard import SummaryWriter
import os
import numpy as np
import glob
import torchvision
import torch
import torch.optim as optim
from torch.utils.data import DataLoader
import torch.nn as nn
from torch.utils.data import Dataset
import matplotlib.pyplot as plt
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import torch.functional as F
import torch.autograd as autograd
drive.mount('/content/drive', force_remount=True)
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

dataset 전처리 및 로드 코드

In [ ]:
class data_load(Dataset):

    def __init__(self, input_path):
        self.data_file_list = getFileNames(input_path)
        # 

    def __len__(self):
        return len(self.data_file_list)

    def __getitem__(self, index):
        self.data = readDataFile(self.data_file_list[index])
        self.label = parseFileName(self.data_file_list[index])
        return self.data, self.label

def getFileNames(input_path):
    input_path = glob.glob('%s/*.csv' % input_path)

    csv_list = []
    for i in range(len(input_path)):
        contents = np.genfromtxt(input_path[i], delimiter=' ')
        data = list(contents[20:,1:])
        if len(data) <= 2000:
            csv_list.append(input_path[i])

    return csv_list

def parseFileName(fName):
    # gestureId_list = []

    # for fName in dataFile:
    fName = fName.split("/")[-1]
    noSuffix = fName.split(".")[0]
    fields = noSuffix.split("_")	
    gestureId = fields[2]				
    if (gestureId[-1] == 'A'):
        twoModalities = True
        gestureId = gestureId[:-1]

    # gestureId_list.append(int(gestureId)-1)
    # gestureId_list = np.array(gestureId_list)
    return torch.tensor(int(gestureId)-1)

def readDataFile(dataFile):
    # data_all = []
    
    # for m in range(len(dataFile)):
    contents = np.genfromtxt(dataFile, delimiter=' ')
    data = list(contents[20:,1:])

    data_list = []

    for i in range(len(data)):
        data_line = []
        
        if data[i][0] == 0 and data[i][1] == 0 and data[i][2] == 0:
            continue 
        while len(data[i]) != 0:
            for _ in range(3):
                data_line.append(data[i][0])
                data[i] = np.delete(data[i],0)
            data[i] = np.delete(data[i],0)
        data_list.append(data_line)

        # data_all.append(torch.tensor(data_list))
    
    # data_all = torch.nn.utils.rnn.pad_sequence(data_all)
    # print(data_all.size())
    # pad = [0]*3
    
    # for i in range(len(data_list)):
    #     if len(data_list) < 2000:
    #         for _ in range(2000-len(data_list)):
    #             data_list.append(pad)
    # if len(data_list) != 2000:
    #     print(dataFile)
    # data_list = np.array(data_list)

    index = []
    A = len(data_list) // 100
    for i in range(0,len(data_list),A):
        if len(index) == 100:
            break
        index.append(data_list[i])
    data_list = index
    return torch.tensor(data_list)

사용 model 코드

In [ ]:
class RNN_MODEL(nn.Module):
    def __init__(self, input_size, hidden_size, batch_size, sequence_length, num_layers, num_classes, device):
        super(RNN_MODEL, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, dropout=0.2, batch_first=True)
        self.classifier = nn.Sequential(
            nn.Linear(self.hidden_size,num_classes)
        )
        self.device = device
    
    def forward(self, x):
        # Set initial hidden and cell states
        
       
        # packed_input = torch.nn.utils.rnn.pack_padded_sequence(x, input_lengths.tolist(), batch_first=True)


        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(self.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(self.device)
        # Forward propagate LSTM

        out, (ht, ct) = self.lstm(x, (h0, c0))
          # out: tensor of shape (batch_size, seq_length, hidden_size)
        # Decode the hidden state of the last time step
        # out = out.contiguous().view(len(out), sequence_length*hidden_size)
        
        out = self.classifier(ht[-1])
        
        return out

hyperparameter 조정

In [ ]:
# Hyper-parameters
sequence_length = 2000
input_size = 60
hidden_size = 30
num_layers = 1
num_classes = 12
batch_size = 1
num_epochs = 20
learning_rate = 0.01
device = torch.device('cuda')
train_path = '/content/drive/My Drive/human_motion/traindata'
test_path = '/content/drive/My Drive/human_motion/testdata'
net = RNN_MODEL(input_size, hidden_size, batch_size, sequence_length, num_layers, num_classes, device)
net.to(device)
#net.load_state_dict(torch.load('/content/drive/My Drive/human_motion/model/test_acc_0.8608695652173913.pt'))

dataset load 코드

In [ ]:
train_dataset = data_load(train_path)
test_dataset = data_load(test_path)
train_loader = DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)

main 코드

In [ ]:
criterion = nn.CrossEntropyLoss().to(device)
optimizer = optim.Adam(net.parameters(), lr=0.001)

total_step = len(train_loader)

max_test_acc = 0.7217
for epoch in range(num_epochs):
    net.train()
    train_correct = 0
    train_total = 0
    loss_arr = []
    acc_arr = []
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device,dtype=torch.float32)
        labels = labels.to(device)
        # labels = labels.reshape(batch_size,1)
        # Forward pass
        print(torch.cuda.is_available())
        if i%5 == 0 :
            !nvidia-smi
        outputs = net(images)
        loss = criterion(outputs, labels)
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        loss_arr+=[loss.item()]
        _, predicted = torch.max(outputs.data, 1)
        train_total += labels.size(0)
        train_correct += (predicted == labels).sum().item()
    
    train_accuracy = train_correct/train_total
    # Test the net
    net.eval()
    with torch.no_grad():
        test_correct = 0
        test_total = 0
        for images, labels in test_loader:
            images = images.to(device,dtype=torch.float32)
            labels = labels.to(device)
            outputs = net(images)
            test_loss = criterion(outputs,labels)
            _, predicted = torch.max(outputs.data, 1)
            test_total += labels.size(0)
            test_correct += (predicted == labels).sum().item()
        test_accuracy = test_correct/test_total
    
    if test_accuracy > max_test_acc:
        max_test_acc = test_accuracy
        torch.save(net.state_dict(), '/content/drive/My Drive/human_motion/model/test_acc_{}.pt'.format(max_test_acc))
        
    print('Epoch [{}/{}] \t train_Loss: {:.4f}, train_Accuracy: {:.4f} \t test_Loss: {:.4f}, test_Accuracy: {:.4f}' .format(epoch+1, num_epochs, loss.item(), train_accuracy ,test_loss.item(), test_accuracy))
     


Epoch [1/20], Step [114/114], Loss: 2.5605
Epoch [2/20], Step [114/114], Loss: 2.7035
Epoch [3/20], Step [114/114], Loss: 2.2247
Epoch [4/20], Step [114/114], Loss: 1.9691
Epoch [5/20], Step [114/114], Loss: 2.1249
Epoch [6/20], Step [114/114], Loss: 2.2739
Epoch [7/20], Step [114/114], Loss: 2.2131
Epoch [8/20], Step [114/114], Loss: 2.4138
Epoch [9/20], Step [114/114], Loss: 1.9373
Epoch [10/20], Step [114/114], Loss: 2.0335
Epoch [11/20], Step [114/114], Loss: 1.7445
Epoch [12/20], Step [114/114], Loss: 1.8453
Epoch [13/20], Step [114/114], Loss: 1.5977
Epoch [14/20], Step [114/114], Loss: 1.7364
Epoch [15/20], Step [114/114], Loss: 2.2449
Epoch [16/20], Step [114/114], Loss: 1.1055
Epoch [17/20], Step [114/114], Loss: 1.3607
Epoch [18/20], Step [114/114], Loss: 1.1881
Epoch [19/20], Step [114/114], Loss: 1.2884
Epoch [20/20], Step [114/114], Loss: 1.0136

In [ ]:
torch.save(net.state_dict(), '/content/drive/My Drive/human_motion/model/net_epoch_40.ckpt')

In [ ]:
net = RNN_MODEL(input_size, hidden_size, batch_size, sequence_length, num_layers, num_classes, device).to(device)
net.load_state_dict(torch.load('/content/drive/My Drive/human_motion/model/net_epoch_40.ckpt'))

In [ ]:
!nvidia-smi

In [ ]:
print(torch.cuda.is_available())

In [ ]:
!pip3 install http://download.pytorch.org/whl/cu80/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl
!pip3 install torchvision

In [ ]:
!nvidia-smi